In [1]:
import webbrowser
from aocd.models import Puzzle
puzzle = Puzzle(year=2019, day=7)
webbrowser.open(puzzle.url);

In [3]:
from dataclasses import dataclass
from typing import Callable
import numpy as np
from functools import partial

In [206]:
class ProgramStop(Exception):
    def __init__(self, final_memory):
        super().__init__()
        self.state = final_memory

@dataclass
class OpCode:
    name: str
    code: int
    n_args: int
    func: Callable

def exit(computer):
    raise ProgramStop(computer._memory)
    
def arithmetic(computer, op):
    computer.c = op(computer.a, computer.b)

def input_(computer):
    computer.a = computer.inputs.pop(0)

def output(computer):
    computer.outputs.append(computer.a)

def jump_if(computer, condition):
    if condition(computer.a):
        return computer.b

def arithmethic_comparison(computer, condition):
    computer.c = int(condition(computer.a, computer.b))

code_map = {
    op.code: op for op in [
        OpCode(name="add", code=1, n_args=3, func=partial(arithmetic, op=lambda a, b: a + b)),
        OpCode(name="multiply", code=2, n_args=3, func=partial(arithmetic, op=lambda a, b: a * b)),
        OpCode(name="input", code=3, n_args=1, func=input_),
        OpCode(name="output", code=4, n_args=1, func=output),
        OpCode(name="jump-if-true", code=5, n_args=2, func=partial(jump_if, condition=lambda a: a != 0)),
        OpCode(name="jump-if-false",code=6, n_args=2, func=partial(jump_if, condition=lambda a: a == 0)),
        OpCode(name="less than", code=7, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a < b)),
        OpCode(name="equals", code=8, n_args=3, func=partial(arithmethic_comparison, condition=lambda a, b: a == b)),
        OpCode(name="exit", code=99, n_args=0, func=exit)
    ]
}

def parse_opcode(code: int):
    """
    Parse the opcode and the parameter modes from the given opcode integer
    
    >>> parse_opcode(1002)
    OpCode('multiplay'), [0, 1, 0]
    
    >>> parse_opcode(1107)
    OpCode('less than'), [1, 1, 0]
    """
    instruction = code_map[code % 100]
    modes = [code // 10**(p+2) % 10 for p in range(instruction.n_args)]
    return instruction, modes

In [207]:
class Computer:
    def __init__(self, memory, inputs=None, debug=False):
        self.original_memory = memory
        self.inputs = [] if inputs is None else inputs
        self.debug = debug
    
    def simulate(self):
        self._memory = self.original_memory.copy()
        self.outputs = []
        try:
            self._ip = 0  # instruction pointer
            self._simulate()
            raise ValueError("Program did not terminate and instruction pointer ran out of bounds")
        except ProgramStop as stop:
            if self.debug:
                print("Program Terminated!")
            self._ip = None
            self._final_memory = stop.state
        return self.outputs
    
    def _simulate(self):
        while self._ip < len(self._memory):
            self._instruction, self._param_modes = parse_opcode(self._memory[self._ip])
            self._params = self._memory[self._ip+1 : self._ip+1 + self._instruction.n_args]
            self._log_instruction_call_before()
            new_ip = self._instruction.func(self)
            self._log_instruction_call_after(new_ip)
            if new_ip is None:
                self._ip += 1 + self._instruction.n_args
            else:
                self._ip = new_ip
    
    def start_or_continue(self):
        if hasattr(self, '_ip') and self._ip is not None:
            self._simulate()
        else:
            self.simulate()
    
    # easy accessors for the parameters of the current instruction
    def _get_param(self, index):
        assert len(self._params) > index
        if self._param_modes[index] == 0:
            return self._memory[self._params[index]]
        return self._params[index]
    
    def _set_param(self, index, value):
        assert len(self._params) > index
        assert self._param_modes[index] == 0
        self._memory[self._params[index]] = value
    
    @property
    def a(self):
        return self._get_param(0)
    
    @a.setter
    def a(self, val):
        self._set_param(0, val)
    
    @property
    def b(self):
        return self._get_param(1)
    
    @b.setter
    def b(self, val):
        self._set_param(1, val)
    
    @property
    def c(self):
        return self._get_param(2)
    
    @c.setter
    def c(self, val):
        self._set_param(2, val)
    
    def _log_instruction_call_before(self):
        if not self.debug:
            return
        instr = ' '.join(map(str, self._memory[self._ip:self._ip+1 + self._instruction.n_args]))
        params = [f"[{par}]={self._memory[par]}" if m == 0 else str(par) for par, m in zip(self._params, self._param_modes)]
        print(f"{str(self._ip) + '  ':.<6} {instr+'  ':.<20}  {self._instruction.name+'  ':.<15}  {', '.join(params) + '  ':.<40}", end="")
    
    def _log_instruction_call_after(self, new_ip):
        if not self.debug:
            return
        writeable_params = set([par for par, m in zip(self._params, self._param_modes) if m == 0])
        values = [f"mem[{par}] = {self._memory[par]}" for par in writeable_params]
        if new_ip:
            print(f"✓ Instruction Pointer changed: New value {new_ip}")
        elif values:
            print(f"✓ Result: {', '.join(values)}")
        else:
            print("✓ No effect to memory or execution")

## Example

In [208]:
from itertools import permutations

def get_max_permutation(program):
    curr_max = 0
    curr_perm = None
    for perm in permutations(range(5)):
        signal = 0
        for phase in perm:
            signal = Computer(program, inputs=[phase, signal]).simulate()[0]
        if signal >= curr_max:
            curr_max = signal
            curr_perm = perm
    return curr_max, curr_perm

In [43]:
program = np.array([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0])
get_max_permutation(program)

(43210, (4, 3, 2, 1, 0))

## Part 1

In [46]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
signal, perm = get_max_permutation(program)

In [47]:
signal

30940

In [48]:
puzzle.answer_a = signal

That's the right answer!  You are one gold star closer to rescuing Santa. [Continue to Part Two]


## Part 2

#### Tests

In [222]:
def get_max_permutation2(program):
    curr_max = 0
    curr_perm = None
    
    for perm in permutations(range(5, 10)):
        signal = 0
        computers = [Computer(program, inputs=[phase]) for phase in perm]
        last_computer = computers[-1]
        computers[0].inputs.append(0)
        curr_c = 0
        while len(computers) > 0:
            next_c = (curr_c + 1) % len(computers)
            try:
                computers[curr_c].start_or_continue()
            except IndexError:  # current computer needs more input to continue
                computers[next_c].inputs.append(computers[curr_c].outputs[0])
                computers[curr_c].outputs.clear()
                curr_c = next_c
            except ProgramStop:
                computers[next_c].inputs.append(computers[curr_c].outputs[0])
                del computers[curr_c]  # don't clear outputs, we need the last output
                # don't increase curr_c since we removed from the list
        signal = last_computer.outputs[0]
        if signal >= curr_max:
            curr_max = signal
            curr_perm = perm
    return curr_max, curr_perm

In [224]:
program = np.array([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,
27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])
get_max_permutation2(program)

(139629729, (9, 8, 7, 6, 5))

In [226]:
program = np.array(list(map(int, puzzle.input_data.split(","))))
signal, perm = get_max_permutation2(program)

In [228]:
signal

76211147

In [229]:
puzzle.answer_b = 76211147

That's the right answer!  You are one gold star closer to rescuing Santa.You have completed Day 7! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
